# Prompt Design for Fredholm Equation Solving

This notebook explores different prompting strategies for LLM-based solving of Fredholm integral equations.

## Prompting Styles
1. **Basic**: Direct question-answer format
2. **Chain-of-Thought**: Step-by-step reasoning
3. **Few-Shot**: Learning from examples
4. **Tool-Assisted**: Using computational tools

In [2]:
# Setup
import sys
from pathlib import Path

# Derive project root similar to explore_data notebook
notebook_dir = Path.cwd()
if notebook_dir.name == "notebooks":
    project_root = notebook_dir.parent
else:
    project_root = notebook_dir
    while project_root.parent != project_root:
        if (project_root / "pyproject.toml").exists():
            break
        project_root = project_root.parent

sys.path.insert(0, str(project_root))

# Import prompt modules
from src.prompts import (
    EquationData,
    create_prompt_style,
    create_edge_case_mode,
)

AVAILABLE_STYLES = ["basic", "chain-of-thought", "few-shot", "tool-assisted"]
EDGE_CASE_MODES = ["none", "guardrails", "hints"]

print("Prompt Design Notebook Ready")
print(f"Project root: {project_root}")
print("Styles:", ", ".join(AVAILABLE_STYLES))
print("Edge case modes:", ", ".join(EDGE_CASE_MODES))

Prompt Design Notebook Ready
Project root: c:\Users\ammah\Documents\GitHub\fred-llm
Styles: basic, chain-of-thought, few-shot, tool-assisted
Edge case modes: none, guardrails, hints


## 1. Chain-of-Thought Example

In [3]:
def display_prompt(prompt_text: str, title: str = "Generated Prompt"):
    """Print a prompt with a simple header."""
    print(f"--- {title} ---")
    print(prompt_text)
    print("--- end ---\n")

# Sample equation data
sample_equation = {
    "kernel": "exp(-x*t)",
    "f": "sin(x)",
    "lambda_val": 0.5,
    "a": 0,
    "b": 1,
    "solution": "cos(x) + x",
    "equation_id": "sample_1",
}

# Create EquationData
eq = EquationData(
    u=sample_equation["solution"],
    f=sample_equation["f"],
    kernel=sample_equation["kernel"],
    lambda_val=sample_equation["lambda_val"],
    a=sample_equation["a"],
    b=sample_equation["b"],
    equation_id=sample_equation["equation_id"],
)

# Generate chain-of-thought prompt
style = create_prompt_style("chain-of-thought")
prompt_obj = style.generate(eq, format_type="infix", include_ground_truth=True)

print("Chain-of-Thought Prompt (step-by-step reasoning):")
display_prompt(prompt_obj.prompt, "Chain-of-Thought")

Chain-of-Thought Prompt (step-by-step reasoning):
--- Chain-of-Thought ---
You are an expert mathematician specializing in integral equations.
Your task is to solve Fredholm integral equations of the second kind.

The general form is: u(x) - λ ∫_a^b K(x, t) u(t) dt = f(x)

Approach each problem systematically:
1. Identify the kernel K(x, t), the known function f(x), and the parameter λ
2. Determine the type of kernel (separable, symmetric, etc.)
3. Choose an appropriate solution method
4. Apply the method step by step
5. Verify the solution satisfies the original equation
6. Present the final solution u(x)

Show your reasoning at each step.

Solve the following Fredholm integral equation step by step:

u(x) - 0.5 * ∫_0^1 exp(-x*t) * u(t) dt = sin(x)

Domain: [0, 1]

Please show your complete reasoning process.
--- end ---



## 2. Compare All Prompt Styles

In [4]:
import pandas as pd

print("Compare Prompt Styles")

comparison_data = []
for style_name in ["basic", "chain-of-thought", "few-shot", "tool-assisted"]:
    style = create_prompt_style(style_name)
    eq = EquationData(
        u="u(x)",
        f="x^2",
        kernel="x*t",
        lambda_val=1.0,
        a=0,
        b=1,
        equation_id=f"demo_{style_name}",
)
    prompt_obj = style.generate(eq, format_type="infix", include_ground_truth=False)
    prompt_text = prompt_obj.prompt
    
    comparison_data.append({
        "Style": style_name,
        "Length": len(prompt_text),
        "Lines": prompt_text.count("\n") + 1,
    })
    
    display_prompt(prompt_text, f"{style_name.title()} Style")

df_compare = pd.DataFrame(comparison_data)
print("Style Comparison Table:")
print(df_compare)

Compare Prompt Styles
--- Basic Style ---
You are an expert mathematician specializing in integral equations.
Given a Fredholm integral equation of the second kind, find the solution u(x).

The general form is: u(x) - λ ∫_a^b K(x, t) u(t) dt = f(x)

Provide your answer as a mathematical expression for u(x).

Solve the following Fredholm integral equation:

u(x) - 1.0 * ∫_0^1 x*t * u(t) dt = x^2

Domain: [0, 1]

Provide the solution u(x).
--- end ---

--- Chain-Of-Thought Style ---
You are an expert mathematician specializing in integral equations.
Your task is to solve Fredholm integral equations of the second kind.

The general form is: u(x) - λ ∫_a^b K(x, t) u(t) dt = f(x)

Approach each problem systematically:
1. Identify the kernel K(x, t), the known function f(x), and the parameter λ
2. Determine the type of kernel (separable, symmetric, etc.)
3. Choose an appropriate solution method
4. Apply the method step by step
5. Verify the solution satisfies the original equation
6. Present

## 3. Edge Case Modes

In [5]:
print("Edge Case Modes")
print("Handle special equation conditions with specific guidance.")

for mode_name in ["ill-posed", "separable", "degenerate"]:
    edge_mode = create_edge_case_mode("hints")
    style = create_prompt_style("chain-of-thought", edge_case_mode=edge_mode)
    
    eq = EquationData(
        u="u(x)",
        f="x^2",
        kernel="x*t",
        lambda_val=1.0,
        a=0,
        b=1,
        equation_id=f"edge_{mode_name}",
)
    
    prompt_obj = style.generate(eq, format_type="infix", include_ground_truth=False)
    print(f"Edge Case: {mode_name.title()}")
    display_prompt(prompt_obj.prompt, f"Edge Case: {mode_name.title()}")

Edge Case Modes
Handle special equation conditions with specific guidance.
Edge Case: Ill-Posed
--- Edge Case: Ill-Posed ---
You are an expert mathematician specializing in integral equations.
Your task is to solve Fredholm integral equations of the second kind.

The general form is: u(x) - λ ∫_a^b K(x, t) u(t) dt = f(x)

Approach each problem systematically:
1. Identify the kernel K(x, t), the known function f(x), and the parameter λ
2. Determine the type of kernel (separable, symmetric, etc.)
3. Choose an appropriate solution method
4. Apply the method step by step
5. Verify the solution satisfies the original equation
6. Present the final solution u(x)

Show your reasoning at each step.
Note: If no closed-form solution exists, state so. If only numerical approximation is possible, indicate that.

Solve the following Fredholm integral equation step by step:

u(x) - 1.0 * ∫_0^1 x*t * u(t) dt = x^2

Domain: [0, 1]

Please show your complete reasoning process.
--- end ---

Edge Case: Se

## 4. Solution Format Types

In [6]:
print("Solution Format Types")
print("Customize expected output format.")

solution_types = ["symbolic", "numeric", "both"]

for sol_type in solution_types:
    style = create_prompt_style("basic")
    eq = EquationData(
        u="u(x)",
        f="x^2",
        kernel="x*t",
        lambda_val=1.0,
        a=0,
        b=1,
        equation_id=f"sol_{sol_type}",
)
    
    prompt_obj = style.generate(eq, format_type="infix", include_ground_truth=False)
    print(f"Solution Type: {sol_type}")
    display_prompt(prompt_obj.prompt, f"Solution Type: {sol_type}")

Solution Format Types
Customize expected output format.
Solution Type: symbolic
--- Solution Type: symbolic ---
You are an expert mathematician specializing in integral equations.
Given a Fredholm integral equation of the second kind, find the solution u(x).

The general form is: u(x) - λ ∫_a^b K(x, t) u(t) dt = f(x)

Provide your answer as a mathematical expression for u(x).

Solve the following Fredholm integral equation:

u(x) - 1.0 * ∫_0^1 x*t * u(t) dt = x^2

Domain: [0, 1]

Provide the solution u(x).
--- end ---

Solution Type: numeric
--- Solution Type: numeric ---
You are an expert mathematician specializing in integral equations.
Given a Fredholm integral equation of the second kind, find the solution u(x).

The general form is: u(x) - λ ∫_a^b K(x, t) u(t) dt = f(x)

Provide your answer as a mathematical expression for u(x).

Solve the following Fredholm integral equation:

u(x) - 1.0 * ∫_0^1 x*t * u(t) dt = x^2

Domain: [0, 1]

Provide the solution u(x).
--- end ---

Solution

## 5. Expression Format Types

In [7]:
print("Expression Format Types")
print("Different mathematical notation formats.")

format_types = ["latex", "infix", "rpn", "python"]

for fmt_type in format_types:
    style = create_prompt_style("basic")
    eq = EquationData(
        u="u(x)",
        f="sin(x) + cos(x)",
        kernel="exp(-x*t)",
        lambda_val=0.5,
        a=0,
        b="pi",
        equation_id=f"fmt_{fmt_type}",
)
    
    prompt_obj = style.generate(eq, format_type=fmt_type, include_ground_truth=False)
    print(f"Format: {fmt_type}")
    display_prompt(prompt_obj.prompt, f"Format: {fmt_type}")

Expression Format Types
Different mathematical notation formats.
Format: latex
--- Format: latex ---
You are an expert mathematician specializing in integral equations.
Given a Fredholm integral equation of the second kind, find the solution u(x).

The general form is: u(x) - λ ∫_a^b K(x, t) u(t) dt = f(x)

Provide your answer as a mathematical expression for u(x).

Solve the following Fredholm integral equation:

u(x) - 0.5 * ∫_0^pi exp(-x\cdot t) * u(t) dt = sin(x) + cos(x)

Domain: [0, pi]

Provide the solution u(x).
--- end ---

Format: infix
--- Format: infix ---
You are an expert mathematician specializing in integral equations.
Given a Fredholm integral equation of the second kind, find the solution u(x).

The general form is: u(x) - λ ∫_a^b K(x, t) u(t) dt = f(x)

Provide your answer as a mathematical expression for u(x).

Solve the following Fredholm integral equation:

u(x) - 0.5 * ∫_0^pi exp(-x*t) * u(t) dt = sin(x) + cos(x)

Domain: [0, pi]

Provide the solution u(x).
--- e

## 6. Few-Shot Learning Examples

In [8]:
print("Few-Shot Learning")
print("Provide examples to guide the model.")

few_shot_style = create_prompt_style("few-shot")

eq = EquationData(
    u="u(x)",
    f="x^2",
    kernel="x*t",
    lambda_val=1.0,
    a=0,
    b=1,
    equation_id="few_shot_demo",
)

prompt_obj = few_shot_style.generate(eq, format_type="infix", include_ground_truth=False)
display_prompt(prompt_obj.prompt, "Few-Shot")

Few-Shot Learning
Provide examples to guide the model.
--- Few-Shot ---
You are an expert mathematician specializing in integral equations.
I will show you examples of solved Fredholm integral equations, then ask you to solve a new one.

The general form is: u(x) - λ ∫_a^b K(x, t) u(t) dt = f(x)

Example 1:
u(x) - ∫_0^1 x*t * u(t) dt = x

Solution:
Let me solve this step by step.

The kernel K(x,t) = x*t is separable, so I can write:
∫_0^1 x*t * u(t) dt = x * ∫_0^1 t * u(t) dt = x * c

where c = ∫_0^1 t * u(t) dt is a constant.

Substituting back: u(x) = x + x*c = x(1 + c)

To find c, multiply both sides by t and integrate:
c = ∫_0^1 t * u(t) dt = ∫_0^1 t * t(1+c) dt = (1+c) * ∫_0^1 t² dt = (1+c)/3

Solving: c = (1+c)/3 → 3c = 1 + c → 2c = 1 → c = 1/2

Therefore: u(x) = x(1 + 1/2) = 3x/2

Example 2:
u(x) - ∫_0^1 e^(x+t) * u(t) dt = 1

Solution:
The kernel K(x,t) = e^(x+t) = e^x * e^t is separable.

Let c = ∫_0^1 e^t * u(t) dt (a constant)

Then: u(x) = 1 + e^x * c

Substituting to find

## 7. Batch Prompt Generation

In [9]:
import pandas as pd

print("Batch Prompt Generation")
print("Generate prompts for multiple equations.")

batch_style = create_prompt_style("basic")

equations = [
    EquationData(u="u(x)", f="sin(x)", kernel="exp(-x*t)", lambda_val=0.5, a=0, b=1, equation_id="batch_1"),
    EquationData(u="u(x)", f="x^2 + 1", kernel="x*t", lambda_val=1.0, a=0, b=1, equation_id="batch_2"),
    EquationData(u="u(x)", f="exp(-x)", kernel="cos(x-t)", lambda_val=0.25, a=0, b="pi", equation_id="batch_3"),
]

batch_prompts = [batch_style.generate(eq, format_type="infix", include_ground_truth=False) for eq in equations]

df_results = pd.DataFrame(
    [
        {
            "ID": p.metadata.get("equation_id", f"eq_{i+1}"),
            "Kernel": equations[i].kernel,
            "f(x)": equations[i].f,
            "lambda": equations[i].lambda_val,
            "Prompt Length": len(p.prompt),
        }
        for i, p in enumerate(batch_prompts)
    ]
 )

print("Batch summary:")
print(df_results)
print(f"Total prompts generated: {len(batch_prompts)}")

for p in batch_prompts:
    display_prompt(p.prompt, p.metadata.get("equation_id", "batch"))

Batch Prompt Generation
Generate prompts for multiple equations.
Batch summary:
     ID     Kernel     f(x)  lambda  Prompt Length
0  eq_1  exp(-x*t)   sin(x)    0.50            408
1  eq_2        x*t  x^2 + 1    1.00            403
2  eq_3   cos(x-t)  exp(-x)    0.25            411
Total prompts generated: 3
--- batch ---
You are an expert mathematician specializing in integral equations.
Given a Fredholm integral equation of the second kind, find the solution u(x).

The general form is: u(x) - λ ∫_a^b K(x, t) u(t) dt = f(x)

Provide your answer as a mathematical expression for u(x).

Solve the following Fredholm integral equation:

u(x) - 0.5 * ∫_0^1 exp(-x*t) * u(t) dt = sin(x)

Domain: [0, 1]

Provide the solution u(x).
--- end ---

--- batch ---
You are an expert mathematician specializing in integral equations.
Given a Fredholm integral equation of the second kind, find the solution u(x).

The general form is: u(x) - λ ∫_a^b K(x, t) u(t) dt = f(x)

Provide your answer as a mathem

## 8. Summary

In [10]:
print("Prompt Design Summary")
print("Styles: basic, chain-of-thought, few-shot, expert")
print("Edge cases: ill-posed, separable, degenerate")
print("Formats: latex, infix, rpn, python")
print("Done")

Prompt Design Summary
Styles: basic, chain-of-thought, few-shot, expert
Edge cases: ill-posed, separable, degenerate
Formats: latex, infix, rpn, python
Done
